In [259]:
#imports
import numpy as np
import math
import time
import json
import pandas as pd
import requests
from pandas import json_normalize
import os
import csv
pd.options.mode.chained_assignment = None
import matplotlib.pyplot as plt
import seaborn as sns

In [67]:
def season(i):
    df = pd.read_csv("Season"+str(i)+".csv", dtype='unicode')
    return df

In [77]:
def locationextractor(df,typename):
    if(typename=='pass'):
        df = df.query("(type_name=='Pass')&(team_name=='Barcelona')&\
        (pass_outcome_name not in ['Incomplete','Injury Clearance','Pass Offside','Out','Unknown'])\
        &(pass_type_name not in ['Free Kick', 'Corner', 'Throw-in', 'Kick Off'])").reset_index(drop=True)
    else:
        df = df.query("(type_name=='Carry')&(team_name=='Barcelona')").reset_index(drop=True)
    type_name_end = typename+'_end_location'
    df['location'] = df['location'].astype(str).str.strip('[]')
    df['y_loc'] = df['location'].str.split(', ').str.get(0).str.strip().astype(float)
    df['x_loc'] = df['location'].str.split(', ').str.get(1).str.strip().astype(float)
    df[type_name_end] = df[type_name_end].astype(str).str.strip('[]')
    df['end_y_loc'] = df[type_name_end].str.split(', ').str.get(0).str.strip().astype(float)
    df['end_x_loc'] = df[type_name_end].str.split(', ').str.get(1).str.strip().astype(float)
    return df

In [104]:
def season_location(df):
    locs = np.zeros(16)
    locs[0] = len(df.query("(y_loc<=40)&(end_y_loc<=40)"))
    locs[1] = len(df.query("(y_loc<=40)&(end_y_loc<80)&(end_y_loc>40)"))
    locs[2] = len(df.query("(y_loc<=40)&(end_y_loc>=80)"))
    locs[3] = len(df.query("(y_loc>40)&(y_loc<80)&(end_y_loc<=40)"))
    locs[4] = len(df.query("(y_loc>40)&(y_loc<80)&(end_y_loc<80)&(end_y_loc>40)"))
    locs[5] = len(df.query("(y_loc>40)&(y_loc<80)&(end_y_loc>=80)"))
    locs[6] = len(df.query("(end_y_loc<=40)&(y_loc>=80)"))
    locs[7] = len(df.query("(y_loc>=80)&(end_y_loc<80)&(end_y_loc>40)"))
    locs[8] = len(df.query("(y_loc>=80)&(end_y_loc>=80)"))
    locs[9] = len(df.query("(y_loc<=40)&(end_y_loc>=102)&(end_y_loc<=120)&(end_x_loc>=18)&(end_x_loc<=62)"))
    locs[10] = len(df.query("(y_loc<80)&(y_loc>40)&(end_y_loc>=102)&(end_y_loc<=120)&(end_x_loc>=18)&(end_x_loc<=62)"))
    locs[11] = len(df.query("(y_loc>=80)&(end_y_loc>=102)&(end_y_loc<=120)&(end_x_loc>=18)&(end_x_loc<=62)"))
    locs[12] = len(df.query("(x_loc>=18)&(x_loc<=62)&(y_loc>80)&(y_loc<102)&\
                            (end_y_loc>=102)&(end_y_loc<=120)&(end_x_loc>=18)&(end_x_loc<=62)"))
    locs[13] = len(df.query("(x_loc>=18)&(x_loc<=62)&(y_loc>80)&(y_loc<102)&\
                            (end_y_loc>=80)&(end_y_loc<=120)&((end_x_loc<18)or(end_x_loc>62))"))
    locs[14] = len(df.query("(x_loc>=18)&(x_loc<=62)&(y_loc>80)&(y_loc<102)&(end_y_loc<80)"))
    locs[15] = len(df)
    return locs

In [105]:
#Calculating for all seasons
passloc = []
carryloc = []

for i in range(11):
    df = season(i)
    passdf = locationextractor(df,'pass')
    carrydf = locationextractor(df,'carry')
    passloc.append(season_location(passdf))
    carryloc.append(season_location(carrydf))
    
passloc = np.asarray(passloc)
carryloc = np.asarray(carryloc)

In [118]:
#zone dataframes
zone2zone = ['D2D','D2M','D2F','M2D','M2M','M2F','F2D','F2M','F2F','D2P','M2P','F2P','ZF2P','ZF2W','ZF2B','Total']
seasonspassdf = pd.DataFrame(data=passloc,columns=zone2zone)
seasonscarrydf = pd.DataFrame(data=carryloc,columns=zone2zone)
seasons = ['08/09','09/10','10/11','11/12','12/13','13/14','14/15','15/16','16/17','17/18','18/19']

In [254]:
# Fraction plots
# fig, axs = plt.subplots(2,2,figsize=(20,15))
# ax=axs[0,0]
# ax.plot(seasons,(seasonspassdf['D2M']+seasonspassdf['D2F'])/(seasonspassdf['D2D']+seasonspassdf['D2M']+seasonspassdf['D2F']),\
#     seasons,(seasonscarrydf['D2M']+seasonscarrydf['D2F'])/(seasonscarrydf['D2D']+seasonscarrydf['D2M']+seasonscarrydf['D2F']))
# ax.set_ylabel("Fraction of passes/carries")
# ax.set_xticklabels(seasons,rotation=90)
# ax.legend(("Passes", "Carries"))
# ax.set_title("Fraction of passes/carries starting from defensive 3rd, ending in the middle/final 3rd")
# ax=axs[0,1]
# ax.plot(seasons,(seasonspassdf['M2F'])/(seasonspassdf['M2D']+seasonspassdf['M2M']+seasonspassdf['M2F']),\
#         seasons,(seasonscarrydf['M2F'])/(seasonscarrydf['M2D']+seasonscarrydf['M2M']+seasonscarrydf['M2F']))
# ax.set_ylabel("Fraction of passes/carries")
# ax.set_xticklabels(seasons,rotation=90)
# ax.legend(("Passes", "Carries"))
# ax.set_title("Fraction of passes/carries starting from middle 3rd, ending in the final 3rd")
# ax=axs[1,0]
# ax.plot(seasons,(seasonspassdf['F2P'])/(seasonspassdf['F2D']+seasonspassdf['F2M']+seasonspassdf['F2F']),\
#         seasons,(seasonscarrydf['F2P'])/(seasonscarrydf['F2D']+seasonscarrydf['F2M']+seasonscarrydf['F2F']))
# ax.set_ylabel("Fraction of passes/carries")
# ax.set_xticklabels(seasons,rotation=90)
# ax.legend(("Passes", "Carries"))
# ax.set_title("Fraction of total passes/carries starting from final 3rd, ending in opposition box")
# ax=axs[1,1]
# ax.plot(seasons,(seasonspassdf['ZF2P'])/(seasonspassdf['ZF2P']+seasonspassdf['ZF2W']+seasonspassdf['ZF2B']),\
#         seasons,(seasonscarrydf['ZF2P'])/(seasonscarrydf['ZF2P']+seasonscarrydf['ZF2W']+seasonscarrydf['ZF2B']))
# ax.set_ylabel("Fraction of passes/carries")
# ax.set_xticklabels(seasons,rotation=90)
# ax.legend(("Passes", "Carries"))
# ax.set_title("Fraction of total passes/carries starting from Zone 14, ending in opposition box")
# fig.savefig("Fractions.png")

In [167]:
def zone14(i):
    df = season(i)
    df['time'] = df['minute'].astype(float)*60+df['second'].astype(float)
    df['possession'] = df['possession'].astype(float) + df.match_id.astype(float)*1000
    passdf = locationextractor(df,'pass')
    carrydf = locationextractor(df,'carry')
    passdf = passdf[['possession','time','x_loc','y_loc','end_x_loc','end_y_loc']]
    carrydf = carrydf[['possession','time','x_loc','y_loc','end_x_loc','end_y_loc']]
    movedf = passdf.copy()
    movedf = movedf.append(carrydf)
    movedf = movedf.query("(x_loc>=18)&(x_loc<=62)&(y_loc>80)&(y_loc<102)&\
                            (end_y_loc>=102)&(end_y_loc<=120)&(end_x_loc>=18)&(end_x_loc<=62)")
    poss = movedf.possession.unique().tolist()
    counter = 0
    for p in poss:
        time1 = movedf.query("(possession==@p)")['time'].unique().tolist()
        for t in time1:
            temp = df.query("(possession==@p)&(time>@t)&(time<(@t+2))")
            if('Shot' in temp.type_name.unique().tolist()):
                counter = counter+1
    return len(movedf),counter

In [178]:
def pc2p(i):
    df = season(i)
    df['time'] = df['minute'].astype(float)*60+df['second'].astype(float)
    df['possession'] = df['possession'].astype(float) + df.match_id.astype(float)*1000
    passdf = locationextractor(df,'pass')
    carrydf = locationextractor(df,'carry')
    passdf = passdf[['possession','time','x_loc','y_loc','end_x_loc','end_y_loc']]
    carrydf = carrydf[['possession','time','x_loc','y_loc','end_x_loc','end_y_loc']]
    movedf = passdf.copy()
    movedf = movedf.append(carrydf)
    movedf = movedf.query("(end_y_loc>=102)&(end_y_loc<=120)&(end_x_loc>=18)&(end_x_loc<=62)")
    poss = movedf.possession.unique().tolist()
    counter = 0
    for p in poss:
        time1 = movedf.query("(possession==@p)")['time'].unique().tolist()
        for t in time1:
            temp = df.query("(possession==@p)&(time>@t)&(time<(@t+2))")
            if('Shot' in temp.type_name.unique().tolist()):
                counter = counter+1
    return len(movedf),counter

In [173]:
#Zone 14 productivity
zone14len = []
immediateshots = []

for i in range(11):
    l1,l2 = zone14(i)
    zone14len.append(l1)
    immediateshots.append(l2)
    
zone14len = np.asarray(zone14len)
immediateshots = np.asarray(immediateshots)

In [179]:
# All passes and carries to penalty area
total2p = []
immshots = []

for i in range(11):
    l1,l2 = pc2p(i)
    total2p.append(l1)
    immshots.append(l2)
    
total2p = np.asarray(total2p)
immshots = np.asarray(immshots)

In [183]:
# Productivity upon reaching penalty area
# fig, ax = plt.subplots(figsize=(10,6))
# ax.plot(seasons,[immediateshots[i]/zone14len[i] for i in range(len(zone14len))],
#        seasons,[immshots[i]/total2p[i] for i in range(len(total2p))])
# ax.set_ylabel("Fraction of passes/carries resulting in shots")
# ax.set_xticklabels(seasons,rotation=90)
# ax.legend(("Origination from zone 14","Originating from anywhere"))
# ax.set_title("Fraction of passes/carries to penalty area, leading to shot within 2 seconds")
# fig.savefig("PenaltyArea.png")

In [186]:
def timetopenalty(i):
    df = season(i)
    df['time'] = df['minute'].astype(float)*60+df['second'].astype(float)
    df['possession'] = df['possession'].astype(float) + df.match_id.astype(float)*1000
    passdf = locationextractor(df,'pass')
    carrydf = locationextractor(df,'carry')
    passdf = passdf[['possession','time','x_loc','y_loc','end_x_loc','end_y_loc']]
    carrydf = carrydf[['possession','time','x_loc','y_loc','end_x_loc','end_y_loc']]
    movedf = passdf.copy()
    movedf = movedf.append(carrydf)
    poss = movedf.possession.unique().tolist()
    def3rd = []
    mid3rd = []
    for p in poss:
        uniquep = movedf.query("(possession==@p)")
        if(uniquep.y_loc.tolist()[0]<=40):
            time1 = uniquep.time.unique().tolist()[0]
            temp = uniquep.query("(end_y_loc>=102)&(end_y_loc<=120)&(end_x_loc>=18)&(end_x_loc<=62)").time.tolist()
            if(temp!=[]):
                time2 = temp[0]
                def3rd.append(time2-time1)
        elif((uniquep.y_loc.tolist()[0]>40)&(uniquep.y_loc.tolist()[0]<80)):
            time3 = uniquep.time.unique().tolist()[0]
            temp = uniquep.query("(end_y_loc>=102)&(end_y_loc<=120)&(end_x_loc>=18)&(end_x_loc<=62)").time.tolist()
            if(temp!=[]):
                time4 = temp[0]
                mid3rd.append(time4-time3)
    return np.median(def3rd),np.median(mid3rd)

In [187]:
#Time spent to reach penalty area from def and mid 3rds
def3rd = []
mid3rd = []

for i in range(11):
    l1,l2 = timetopenalty(i)
    def3rd.append(l1)
    mid3rd.append(l2)
    
def3rd = np.asarray(def3rd)
mid3rd = np.asarray(mid3rd)

In [190]:
# fig, ax = plt.subplots(figsize=(10,6))
# ax.plot(seasons,def3rd,seasons,mid3rd)
# ax.set_ylabel("Median Time")
# ax.set_xticklabels(seasons,rotation=90)
# ax.legend(("From Def 3rd","From Mid 3rd"))
# ax.set_title("Median time to reach penalty area when possessions start from def or mid 3rds")
# fig.savefig("Timing.png")

In [202]:
def counterattacks(i):
    df = season(i)
    df['possession'] = df['possession'].astype(float) + df.match_id.astype(float)*1000
    counters = df.query("(possession_team_name=='Barcelona')&(play_pattern_name=='From Counter')")
    shotdf = df.query("(team_name=='Barcelona')&(type_name=='Shot')")
    countershots = len(shotdf.query("(play_pattern_name=='From Counter')"))
    return countershots,len(counters.possession.unique().tolist())

In [203]:
#Shots and counter-attack shots
countershots = []
counters = []

for i in range(11):
    l1,l2 = counterattacks(i)
    countershots.append(l1)
    counters.append(l2)
    
countershots = np.asarray(countershots)
counters = np.asarray(counters)

In [213]:
games=[31, 35, 33, 37, 32, 31, 38, 33, 34, 36, 34]

In [212]:
# fig, ax = plt.subplots(figsize=(10,6))
# ax.plot(seasons,[counters[i]/games[i] for i in range(11)],seasons,[countershots[i]/games[i] for i in range(11)])
# ax.set_ylabel("Counters per 90")
# ax.set_xticklabels(seasons,rotation=90)
# ax.set_title("Counter-attacking over the seasons")
# ax.legend(("Counters per 90","Counters per 90 resulting in shots"))
# fig.savefig("Counterattacks.png")

In [251]:
def counterpressure(i):
    df = season(i)
    df['time'] = df['minute'].astype(float)*60+df['second'].astype(float)
    df['possession'] = df['possession'].astype(float) + df.match_id.astype(float)*1000
    cpdf = df.query("(possession_team_name!='Barcelona')&(counterpress=='True')")
    successp = 0.0
    shotp = 0.0
    t_elap = []
    for k, row in cpdf.iterrows():
        t1 = row['time']
        tp6 = t1+6
        nextdf = df[(df.time>=t1)&(df.time <= tp6)]
        if('Barcelona' in nextdf.possession_team_name.tolist()):
            successp = successp + 1.0
            t2 = nextdf.query("(possession_team_name=='Barcelona')")['time'].unique().tolist()[0]
            newposs = nextdf.query("(possession_team_name=='Barcelona')")['possession'].unique().tolist()[0]
            shotcp = df.query("(team_name=='Barcelona')&(type_name=='Shot')&(time>@t2)&(time<@t2+7)")
            shotdf = df.query("(possession==@newposs)&(type_name=='Shot')&(team_name=='Barcelona')")
            if(len(shotcp)>0):
                shotp = shotp + 1.0
            if(len(shotdf)>0):
                t3 = shotdf.time.unique().tolist()[0]
                t_elap.append(t3-t2)
    return successp, shotp, t_elap

In [252]:
#Counterpressure productivity
cp = []
cpshot = []
t_elap = []

for i in range(11):
    l1,l2,l3 = counterpressure(i)
    cp.append(l1)
    cpshot.append(l2)
    t_elap.append(l3)
    
cp = np.asarray(cp)
cpshot = np.asarray(cpshot)
t_elap = np.asarray(t_elap)

In [258]:
# fig, ax = plt.subplots(figsize=(10,6))
# ax.plot(seasons,[cpshot[i]/cp[i] for i in range(11)])
# ax.set_ylabel("Successful Counter pressures Fraction")
# ax.set_xticklabels(seasons,rotation=90)
# ax.set_title("Fraction of successful counter pressing leading to immediate shots")
# fig.savefig("Counterpress.png")